# Proyecto: Predicción de Anemia en Mujeres

## 📊 Perfil del Proyecto

## Fase 1. Business Understanding

La anemia en mujeres en edad fértil representa un desafío significativo de salud pública en Perú, con una prevalencia nacional que oscila entre 30-40% según reportes del Ministerio de Salud. La detección temprana es crucial para implementar intervenciones efectivas, pero los métodos actuales de screening presentan limitaciones en cobertura y eficiencia. Este proyecto busca desarrollar un modelo predictivo que permita identificar mujeres en riesgo de anemia utilizando datos disponibles de encuestas nacionales de salud.

### 1. Objetivos
Objetivo Principal:
Desarrollar un modelo de clasificación binaria que prediga la presencia de anemia en mujeres peruanas en edad fértil (15-49 años) con un F1-Score ≥ 0.85.

#### Objetivos Específicos:

1. Identificar los cinco principales factores de riesgo asociados con anemia
2. Establecer un sistema de priorización para intervenciones de salud pública

3. Reducir los costos operativos de screening en al menos 30%

4. Proporcionar recomendaciones accionables para programas de prevención

### Criterios de Éxito
#### Métricos Técnicos:

- F1-Score: ≥ 0.85 en validación cruzada

- Precisión: ≥ 0.80

- Sensibilidad: ≥ 0.85

- AUC-ROC: ≥ 0.90

#### Métricas de Negocio:

- Identificación de al menos 5 factores de riesgo con significancia estadística

- Reducción del 30% en costos de screening mediante focalización

- Tasa de detección temprana del 80% en población de riesgo

- Validación y aceptación por parte de expertos en salud pública

## Fase 2. Data Understanding

### 📥 2. Fuentes de Datos

#### 📁 Descripción del Dataset

El proyecto utiliza dos datasets principales de encuestas de salud peruanas:

1. `RECH0.csv`: Datos demográficos y socioeconómicos de hogares

    - Registros: 37,390

    - Variables: 44 columnas

    - Período de recolección: 2024

2. `RECH5.csv`: Datos de salud y nutrición de mujeres en edad fértil

    - Registros: 38,488

    - Variables: 40 columnas

    - Período de recolección: 2024

3. `RECH23.csv`: Datos de características de vivienda
    
    - Registros: 37,390

    - Variables: 130 columnas

    - Período de recolección: 2024

#### Variables Clave Identificadas
##### Variables Demográficas (RECH0):

- `IDHOGAR`: Identificador único del hogar

- `EDAD_MUJER`: Edad de la mujer en años

- `NIVEL_EDUCATIVO`: Máximo nivel educativo alcanzado

- `REGION`: Región o departamento de residencia

- `AREA_URBANA_RURAL`: Clasificación urbano/rural

- `IDIOMA_MATERNO`: Idioma principal hablado

- `ACCESO_SERVICIOS_BASICOS`: Indicador de acceso a servicios

##### Variables de Salud (RECH5):

- `HEMOGLOBINA`: Nivel de hemoglobina en g/dL

- `IMC`: Índice de Masa Corporal

- `PESO_KG`: Peso en kilogramos

- `TALLA_CM`: Talla en centímetros

- `EMBARAZO_ACTUAL`: Estado de embarazo actual

- `SUPLEMENTOS_HIERRO`: Consumo de suplementos de hierro

- `ANEMIA_CLASIFICACION`: Clasificación actual de anemia

##### Variables de Características de Vivienda (RECH23)

- `FUENTE_AGUA_BEBER`: Fuente principal de abastecimiento de agua para tomar

- `FUENTE_AGUA_HOGAR`: Fuente principal de agua en el hogar

- `PRUEBA_YODO`: Clasificación del estado de yodo en sal

- `TIPO_SERVICIO_HIGIENICO`: Tipo de servicio higienico en el hogar

- `MARCA_SAL_USADA`: Marca de sal usada por el hogar

- `INDICE_RIQUEZA`: Clasificación de índice de riqueza

- `FACTOR_PUNTUACION_RIQUEZA`: Factor de puntuación del índice de riqueza (5 decimales)

#### Estrategia de Integración
Los datasets se unirán mediante la variable IDHOGAR, permitiendo consolidar información demográfica con datos de salud para el análisis integral.

#### Consideraciones Éticas
- Todos los datos personales están anonimizados

- Cumplimiento de la Ley de Protección de Datos Personales peruana

- Uso exclusivo para investigación en salud pública

- Aprobación del comité de ética correspondiente



In [46]:
import pandas as pd
import numpy as np
import os

print('Ruta raíz:', os.getcwd())

df_rech0 = pd.read_csv("md-project/data/raw/968-Modulo1629/RECH0_2024.csv")
df_rech5 = pd.read_csv("md-project/data/raw/968-Modulo1638/RECH5_2024.csv")
df_rech23 = pd.read_csv("md-project/data/raw/968-Modulo1630/RECH23_2024.csv")
df_union = pd.read_csv("md-project/data/RECH0-H5-H23.csv")

import json
from collections import defaultdict

dict_paths = {
    'RECH0': "md-project/data/raw/968-Modulo1629/Diccionario_RECH0.json",
    'RECH5': "md-project/data/raw/968-Modulo1638/Diccionario_RECH5.json",
    'RECH23': "md-project/data/raw/968-Modulo1630/Diccionario_RECH23.json"
}

import json

# def cargar_diccionario_variables(path):
#     """
#     Devuelve un dict {variable: descripcion}. Usa varios posibles campos
#     porque el JSON puede no tener claves uniformes.
#     """
#     with open(path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#     mapping = {}
#     for obj in data.get('variables', []):
#         var = (obj.get('variable') or obj.get('valores') or obj.get('nombre') or "").strip()
#         desc = (obj.get('descripcion') or obj.get('label') or obj.get('obs') or "").strip()
#         if var:
#             # Evita sobrescribir si ya había descripción más larga
#             if var not in mapping or (desc and len(desc) > len(mapping[var])):
#                 mapping[var] = desc if desc else ""
#     return mapping

def cargar_diccionario_variables(refs):
    """
    Acepta:
      - string: ruta a un JSON
      - lista/tupla de rutas
      - clave (o lista de claves) presentes en dict_paths ('RECH0','RECH5','RECH23')
    Devuelve dict {variable: descripcion} combinado.
    Si varias fuentes repiten variable, conserva la descripción más larga.
    """
    # Normalizar a lista
    if isinstance(refs, (str, bytes)):
        refs = [refs]
    elif isinstance(refs, (tuple, set)):
        refs = list(refs)

    rutas = []
    for r in refs:
        if r in dict_paths:          # es una clave
            rutas.append(dict_paths[r])
        else:
            rutas.append(r)          # se asume ya es ruta

    combined = {}
    for ruta in rutas:
        try:
            with open(ruta, 'r', encoding='utf-8') as f:
                data = json.load(f)
        except FileNotFoundError:
            print(f"[WARN] No se encontró: {ruta}")
            continue
        for obj in data.get('variables', []):
            var = (obj.get('variable') or obj.get('valores') or obj.get('nombre') or "").strip()
            desc = (obj.get('descripcion') or obj.get('label') or obj.get('obs') or "").strip()
            if not var:
                continue
            if var not in combined or (desc and len(desc) > len(combined[var])):
                combined[var] = desc
    return combined


Ruta raíz: /opt/notebooks


In [ ]:
# Resumen general Dataset RECH0

# Cargar diccionario específico RECH0
dicc_rech0 = cargar_diccionario_variables('RECH0')

print("=== df_rech0.info() ===")
df_rech0.info()

# Tabla variable + descripción
meta_rech0 = (
    pd.DataFrame({'variable': df_rech0.columns})
      .assign(descripcion=lambda d: d.variable.map(dicc_rech0)
              .fillna("(sin descripción en diccionario)"))
)

print("\n=== Variables y descripciones RECH0 (primeras 40) ===")
display(meta_rech0)

print("\nshape:", df_rech0.shape)

# Head con MultiIndex (solo para visualización)
vista_rech0 = df_rech0.head(10).copy()
vista_rech0.columns = pd.MultiIndex.from_arrays(
    [vista_rech0.columns,
     [dicc_rech0.get(c, "") for c in vista_rech0.columns]],
    names=["variable", "descripcion"]
)
display(vista_rech0)

=== df_rech0.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37390 entries, 0 to 37389
Data columns (total 44 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID1         37390 non-null  int64  
 1   HHID        37390 non-null  int64  
 2   HV000       37390 non-null  object 
 3   HV001       37390 non-null  int64  
 4   HV002       37390 non-null  int64  
 5   HV002A      37390 non-null  int64  
 6   HV003       37390 non-null  int64  
 7   HV004       37390 non-null  int64  
 8   HV007       37390 non-null  int64  
 9   HV008       37390 non-null  int64  
 10  HV009       37390 non-null  int64  
 11  HV010       37390 non-null  int64  
 12  HV011       37390 non-null  int64  
 13  HV012       37390 non-null  int64  
 14  HV013       37390 non-null  int64  
 15  HV014       37390 non-null  int64  
 16  HV015       37390 non-null  int64  
 17  HV017       37390 non-null  int64  
 18  HV020       37390 non-null  int64  
 19  H

,variable,descripcion
0,ID1,Año
1,HHID,Identificacion Cuestionario del Hogar
2,HV000,Código del país
3,HV001,Conglomerado
4,HV002,Vivienda
5,HV002A,Hogar
6,HV003,Número de orden del Informante del cuestionari...
7,HV004,Unidad última de muestreo
8,HV007,Año de entrevista
9,HV008,Fecha de entrevista meses (CMC)



shape: (37390, 44)


variable,ID1,HHID,HV000,HV001,HV002,HV002A,HV003,HV004,HV007,HV008,...,HV043,HV044,UBIGEO,HV022,HV005,CODCCPP,NOMCCPP,LATITUDY,LONGITUDX,NCONGLOME1
descripcion,Año,Identificacion Cuestionario del Hogar,Código del país,Conglomerado,Vivienda,Hogar,Número de orden del Informante del cuestionario del hogar,Unidad última de muestreo,Año de entrevista,Fecha de entrevista meses (CMC),...,Hogares seleccionados para el módulo de mujeres de estado,Selección para el módulo de violencia familiar,Código de Ubicación Gegráfica,Estrato,Factor de ponderación,Codigo de Centro Poblado,Nombre del Centro Poblado,Latitud del conglomerado,Longitud del conglomerado,Número de Conglomerado (proveniente del marco)
0,2024,325502001,PE6,3255,20,1,0,3255,2024,1493,...,0,1,10101,2,0,1,CHACHAPOYAS,-6.225832,-77.861302,7076
1,2024,325503101,PE6,3255,31,1,2,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
2,2024,325503901,PE6,3255,39,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
3,2024,325504001,PE6,3255,40,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
4,2024,325504701,PE6,3255,47,1,2,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
5,2024,325505001,PE6,3255,50,1,1,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
6,2024,325505301,PE6,3255,53,1,1,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076
7,2024,325508901,PE6,3255,89,1,1,3255,2024,1493,...,0,1,10101,2,74497,1,CHACHAPOYAS,-6.225832,-77.861302,7076
8,2024,325509301,PE6,3255,93,1,2,3255,2024,1493,...,0,1,10101,2,607697,1,CHACHAPOYAS,-6.225832,-77.861302,7076


In [48]:
# Resumen general Dataset RECH5
# Cargar diccionario específico RECH5
dicc_rech5 = cargar_diccionario_variables('RECH5')

print("=== df_rech5.info() ===")
df_rech5.info()

# Tabla variable + descripción
meta_rech5 = (
    pd.DataFrame({'variable': df_rech5.columns})
      .assign(descripcion=lambda d: d.variable.map(dicc_rech5)
              .fillna("(sin descripción en diccionario)"))
)

print("\n=== Variables y descripciones RECH5 ===")
display(meta_rech5)

print("\nshape:", df_rech5.shape)

# Head con MultiIndex (solo para visualización)
vista_rech5 = df_rech5.head(10).copy()
vista_rech5.columns = pd.MultiIndex.from_arrays(
    [vista_rech5.columns,
     [dicc_rech5.get(c, "") for c in vista_rech5.columns]],
    names=["variable", "descripcion"]
)
display(vista_rech5)

=== df_rech5.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38488 entries, 0 to 38487
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID1     38488 non-null  int64 
 1   HHID    38488 non-null  int64 
 2   HA0     38488 non-null  int64 
 3   HA1     38488 non-null  int64 
 4   HA2     38488 non-null  int64 
 5   HA3     38488 non-null  int64 
 6   HA4     38488 non-null  int64 
 7   HA5     38488 non-null  int64 
 8   HA6     38488 non-null  int64 
 9   HA11    38488 non-null  int64 
 10  HA12    38488 non-null  int64 
 11  HA12A   38488 non-null  int64 
 12  HA12B   38488 non-null  int64 
 13  HA13    38488 non-null  int64 
 14  HA32    38488 non-null  int64 
 15  HA33    38488 non-null  int64 
 16  HA35    38488 non-null  int64 
 17  HA40    38488 non-null  int64 
 18  HA41    38488 non-null  int64 
 19  HA50    38488 non-null  int64 
 20  HA51    38488 non-null  object
 21  HA52    38488 non-null  int64 
 22

,variable,descripcion
0,ID1,Año
1,HHID,Identificación de caso
2,HA0,Número de orden en hogar
3,HA1,Edad de la mujer en años
4,HA2,Peso en kilogramos (1 decimal)
5,HA3,Talla en centímetros (1 decimal)
6,HA4,Talla/Edad percentil en que se encuentra
7,HA5,Talla/Edad Desviación Estándar
8,HA6,Talla/Edad porcentaje respecto a la mediana de...
9,HA11,Peso/Talla desvio estándar (DHS)



shape: (38488, 40)


variable,ID1,HHID,HA0,HA1,HA2,HA3,HA4,HA5,HA6,HA11,...,HA60,HA61,HA62,HA63,HA64,HA65,HA66,HA67,HA68,HA69
descripcion,Año,Identificación de caso,Número de orden en hogar,Edad de la mujer en años,Peso en kilogramos (1 decimal),Talla en centímetros (1 decimal),Talla/Edad percentil en que se encuentra,Talla/Edad Desviación Estándar,Talla/Edad porcentaje respecto a la mediana de referencia,Peso/Talla desvio estándar (DHS),...,Estado civil,Lea la declaración de consentimiento (VIH),Número de identificación de la muestra de sangre,Resultado de la medición (VIH),Consentimiento para pruebas adicionales,Resultado de la entrevista individual a la mujer,El nivel educativo más alto aprobado de la mujer,El año más alto de educación de la mujer,Nivel educativo más alto (CS para el informe preliminar y final),Peso del VIH (6 decimales)
0,2024,325503101,2,38,704,1525,301,-188,9315,108,...,,,,,,1,2,5,2,
1,2024,325503101,4,15,541,1518,591,-156,9355,16,...,,,,,,1,2,3,2,
2,2024,325503901,2,17,498,1602,2931,-54,9798,-101,...,,,,,,1,2,5,2,
3,2024,325504701,2,32,653,1443,6,-325,8814,146,...,,,,,,1,2,5,2,
4,2024,325505001,1,36,689,1486,57,-253,9077,122,...,,,,,,1,1,6,1,
5,2024,325505301,1,32,729,1570,1303,-112,9590,127,...,,,,,,1,1,6,1,
6,2024,325508901,2,34,573,1533,405,-175,9364,-11,...,,,,,,1,2,5,2,
7,2024,325509301,1,24,669,1547,655,-151,9450,139,...,,,,,,1,3,5,3,
8,2024,325509301,2,18,676,1515,203,-205,9254,171,...,,,,,,1,3,1,3,


In [ ]:
# Resumen general Dataset RECH23
# Cargar diccionario específico RECH23
dicc_rech23 = cargar_diccionario_variables('RECH23')

print("=== df_rech23.info() ===")
df_rech23.info()

# Tabla variable + descripción
meta_rech23 = (
    pd.DataFrame({'variable': df_rech23.columns})
      .assign(descripcion=lambda d: d.variable.map(dicc_rech23)
              .fillna("(sin descripción en diccionario)"))
)

print("\n=== Variables y descripciones RECH23 ===")
display(meta_rech23)

print("\nshape:", df_rech23.shape)

# Head con MultiIndex (solo para visualización)
vista_rech23 = df_rech23.head(10).copy()
vista_rech23.columns = pd.MultiIndex.from_arrays(
    [vista_rech23.columns,
     [dicc_rech23.get(c, "") for c in vista_rech23.columns]],
    names=["variable", "descripcion"]
)
display(vista_rech23)

=== df_rech23.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37390 entries, 0 to 37389
Columns: 130 entries, ID1 to HV271
dtypes: int64(8), object(122)
memory usage: 37.1+ MB

=== Variables y descripciones RECH23 ===


,variable,descripcion
0,ID1,Año
1,HHID,Identificación Cuestionario del Hogar
2,HV201,¿Cuál es la fuente principal de abastecimiento...
3,HV202,¿Cuál es la fuente principal de abastecimiento...
4,HV204,"¿Cuánto tiempo se demora en ir, recoger agua y..."
...,...,...
125,SH227,Prueba de cloro
126,QH227A,La muestra fue tomada por:
127,QH227B,La muestra del agua se extrajo del:
128,HV270,Índice de riqueza



shape: (37390, 130)


variable,ID1,HHID,HV201,HV202,HV204,HV205,HV206,HV207,HV208,HV209,...,SH78,SH79,SH224,SH225U,SH225,SH227,QH227A,QH227B,HV270,HV271
descripcion,Año,Identificación Cuestionario del Hogar,¿Cuál es la fuente principal de abastecimiento de agua para tomar o beber?,¿Cuál es la fuente principal de abastecimiento de agua en su hogar?,"¿Cuánto tiempo se demora en ir, recoger agua y volver?",¿Qué tipo de servicio higiénico tiene su hogar?,¿Tiene electricidad en su hogar?,Su hogar ¿tiene radio?,Su hogar ¿tiene televisor?,Su hogar ¿tiene refrigeradora/congeladora?,...,Algún miembro de su hogar es dueño de tierras agrícolas,Cuántas hectáreas (1 decimal),Prueba de Yodo,Visualización de la bolsa de sal,Marca de sal usada por el hogar,Prueba de cloro,La muestra fue tomada por:,La muestra del agua se extrajo del:,Índice de riqueza,Factor de puntuación del índice de riqueza (5 decimales)
0,2024,325502001,,,,,,,,,...,,,,,,,,,,
1,2024,325503101,11,,996,22,1,0,0,0,...,0,,9,,,2,1,1,2,-.224175006493913
2,2024,325503901,11,,996,11,1,0,0,0,...,0,,9,,,3,1,1,3,.366367930920724
3,2024,325504001,11,,996,22,1,1,0,0,...,0,,9,,,3,1,1,1,-.76953388723092
4,2024,325504701,71,11,996,11,1,1,1,1,...,0,,9,,,5,,,2,.0566849954924239
5,2024,325505001,11,,996,11,1,0,1,0,...,0,,9,,,1,1,1,2,-.256260959044164
6,2024,325505301,11,,996,11,1,0,1,0,...,0,,9,,,3,1,1,2,-.17238395918053
7,2024,325508901,11,,996,11,1,1,0,0,...,0,,9,,,1,1,1,3,.563507474984082
8,2024,325509301,11,,996,11,1,0,1,0,...,0,,9,,,3,1,1,3,.445810583037222


In [45]:
# Resumen general Dataset RECH0-H5-H23
# Cargar diccionario específico RECH0-H5-H23
dicc_union = cargar_diccionario_variables(dict_paths['RECH0','RECH5','RECH23'])

print("=== df_union.info() ===")
df_union.info()

# Tabla variable + descripción
meta_union = (
    pd.DataFrame({'variable': df_union.columns})
      .assign(descripcion=lambda d: d.variable.map(dicc_union)
              .fillna("(sin descripción en diccionario)"))
)

print("\n=== Variables y descripciones REC0-H5-H23 ===")
display(meta_union)

print("\nshape:", df_union.shape)

# Head con MultiIndex (solo para visualización)
vista_union = df_union.head(10).copy()
vista_union.columns = pd.MultiIndex.from_arrays(
    [vista_union.columns,
     [dicc_union.get(c, "") for c in vista_union.columns]],
    names=["variable", "descripcion"]
)
display(vista_union)

KeyError: ('RECH0', 'RECH5', 'RECH23')

### 📊 3. Descriptive analysis

#### 3.1 Análisis descriptivo

In [35]:
# ...existing code...
categorical_cols = [
    # RECH0
    'HV015',  # Resultado de la entrevista en el hogar
    'HV020',  # Muestra de alguna vez en unión
    'HV023',  # Dominio
    'HV024',  # Región
    'HV025',  # Área de residencia
    'HV026',  # Lugar de residencia
    'HV027',  # Selección para hombre/esposo

    # RECH5
    'HA54',   # Actualmente embarazada
    'HA57A',  # Nivel de anemia
    'HA59',   # Nivel de anemia OMS
    'HA52',   # Consentimiento hemoglobina

    # RECH23
    'HV201',  # Fuente principal de agua para beber
    'HV202',  # Fuente principal de agua en el hogar
    'HV205',  # Tipo de servicio higiénico
    'HV206',  # Electricidad en el hogar
    'HV207',  # Radio en el hogar
    'HV208',  # Televisor en el hogar
    'HV209',  # Refrigeradora/congeladora
    'HV210',  # Bicicleta/triciclo
    'HV211',  # Motocicleta/motokar
    'HV212',  # Carro/camión
    'HV213',  # Material del piso
    'HV214',  # Material de paredes exteriores
    # Puedes agregar más según tus necesidades y diccionarios
]

for col in categorical_cols:
    if col in df_3.columns:
        df_3[col] = df_3[col].astype('category')

#Resumen estadístico de todas las variables (describe()).
df_3.describe(include="all").T # incluye categóricas y numéricas

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID1,37390.0,NaN,NaN,NaN,2024.0,0.0,2024.0,2024.0,2024.0,2024.0,2024.0
HHID,37390.0,NaN,NaN,NaN,487870364.241375,94128861.049359,325502001.0,405836876.0,488307601.0,568615926.0,705708801.0
HV201,37390,12,11,22486,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HV202,37390,11,,30710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HV204,37390,31,996,28094,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
SH227,37390,8,3,12943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
QH227A,37390,3,1,18437,NaN,NaN,NaN,NaN,NaN,NaN,NaN
QH227B,37390,7,1,17290,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HV270,37390,6,1,11076,NaN,NaN,NaN,NaN,NaN,NaN,NaN
